In [2]:
import pandas as pd
from datetime import datetime, timedelta, timezone
import numpy as np

import os
%cd ..
from utils import read_json, build_option_expiries
from influxdb_wrapper import InfluxDBWrapper
%cd -

/home/seb/crypto_vol/deribit_volatility_download_and_visualize
/home/seb/crypto_vol/deribit_volatility_download_and_visualize/notebook


In [3]:
config = read_json('../config.json')
bucket = 'btc_vol_surfaces'
wrapper = InfluxDBWrapper(config['database']['url'], config['database']['token'], config['database']['org'], db_timeout=30_000)
obs_time='2023-06-07T06:40:00Z'

In [4]:
start='2023-05-15T00:00:00Z'
end=obs_time
delta=15
tenor='3M'
normalize_by_ATM=False
timeframe='5m'

In [5]:
def set_entries(start, end, delta, tenor='30D', field='mid_iv', normalize_by_ATM=False, timeframe=False):
    historical_volatilities = wrapper.get_historical_risk_reversal_by_delta_and_tenor(bucket, 'volatility',
                                                                                       start,
                                                                                       end,
                                                                                       delta, tenor, field, normalize_by_ATM, timeframe)
    historical_volatilities['rolling_avg'] = historical_volatilities[field].rolling(1*12*24).mean()
    historical_volatilities['rolling_std'] = historical_volatilities[field].rolling(1*12*24).std()

    for idx, row in historical_volatilities.iterrows():
        if row[field] < row['rolling_avg'] - 3 * row['rolling_std']:
            historical_volatilities.loc[idx, 'trade_side'] = 1
        elif row[field] > row['rolling_avg'] + 3 * row['rolling_std']:
            historical_volatilities.loc[idx, 'trade_side'] = -1
        else:
            historical_volatilities.loc[idx, 'trade_side'] = 0

    return historical_volatilities.set_index('timestamp')[field]


entries = set_entries(start='2023-05-15T00:00:00Z', end=obs_time, delta=15, tenor='3M', normalize_by_ATM=False, timeframe='5m')

def set_exits(start, end, delta, tenor='30D', field='mid_iv', normalize_by_ATM=False, timeframe=False):
    historical_volatilities = wrapper.get_historical_risk_reversal_by_delta_and_tenor(bucket, 'volatility',
                                                                                       start,
                                                                                       end,
                                                                                       delta, tenor, field, normalize_by_ATM, timeframe)
    historical_volatilities['rolling_avg'] = historical_volatilities[field].rolling(5*12*24).mean()

    for idx, row in historical_volatilities.iterrows():
        if row[field] > row['rolling_avg']:
            historical_volatilities.loc[idx, 'trade_side'] = -1
        elif row[field] < row['rolling_avg']:
            historical_volatilities.loc[idx, 'trade_side'] = 1
        else:
            historical_volatilities.loc[idx, 'trade_side'] = 0

    return historical_volatilities.set_index('timestamp')[field]


entries = set_entries(start='2023-05-15T00:00:00Z', end=obs_time, delta=15, tenor='3M', normalize_by_ATM=False, timeframe='5m')
exits = set_exits(start='2023-05-15T00:00:00Z', end=obs_time, delta=15, tenor='3M', normalize_by_ATM=False, timeframe='5m')
exits


func:'get_historical_risk_reversal_by_delta_and_tenor' took: 20.9393 sec
func:'get_historical_risk_reversal_by_delta_and_tenor' took: 17.8544 sec
func:'get_historical_risk_reversal_by_delta_and_tenor' took: 17.1577 sec


timestamp
2023-05-15 00:05:00   -0.352679
2023-05-15 00:10:00   -0.586514
2023-05-15 00:15:00   -0.693958
2023-05-15 00:20:00   -0.955055
2023-05-15 00:25:00   -0.909051
                         ...   
2023-06-07 06:20:00    3.278221
2023-06-07 06:25:00    3.250578
2023-06-07 06:30:00    3.134199
2023-06-07 06:35:00    3.110885
2023-06-07 06:40:00    3.092498
Name: mid_iv, Length: 6609, dtype: float64

In [7]:
def RunBacktest_RR(entry, vols_bids, vols_asks, exit_trade, stake_amt, fee, slippage):
    candle_lives = 1
    unrealized_pnl = np.empty(entry.shape[0])
    unrealized_pnl.fill(np.NaN) 
    realized_pnl = np.zeros(entry.shape[0])
    exit_pnl = np.zeros(entry.shape[0])
    
    lowest_pnl_leg_one = 0
    lowest_pnl_leg_two = 0
    
    for index in range(entry.shape[0]):

        if candle_lives != 1:
            candle_lives -= 1
            continue

        if (entry[index] != 0 and entry[index - 1] == 0):

            if entry[index] == 1 and entry[index - 1] == 0:
                vols = vols_asks
            else:
                vols = vols_bids

            entry_price_coin_one = vols[index]
            entry_price_coin_two = vols[index]
            coin_one_amount = 1 #stake_amt / vols[index]
            coin_two_amount = 1 #stake_amt / vols[index]

            candle_lives = 0
            pnl_leg_one = 0
            pnl_leg_two = 0
            while exit_trade[index + candle_lives] == entry[index]:
                #calc unrealized pnl
                if index + candle_lives == entry.shape[0] - 1:
                    px_coin_one_exit = vols[index + candle_lives]
                    px_coin_two_exit = vols[index + candle_lives]
                else:
                    px_coin_one_exit = vols[index + candle_lives]  
                    px_coin_two_exit = vols[index + candle_lives]

                pnl_leg_one = (px_coin_one_exit - vols[index]) * coin_one_amount * -entry[index] \
                    - (fee + slippage) * 2 * entry_price_coin_one * coin_one_amount - (fee + slippage) * 2 * px_coin_one_exit * coin_one_amount
                pnl_leg_two = (px_coin_two_exit - vols[index]) * coin_two_amount * entry[index] \
                    - (fee + slippage) * 2 * entry_price_coin_two * coin_two_amount - (fee + slippage) * 2 * px_coin_two_exit * coin_two_amount
                unrealized_pnl[index + candle_lives] = pnl_leg_one + pnl_leg_two
                
                if pnl_leg_one < lowest_pnl_leg_one:
                    lowest_pnl_leg_one = pnl_leg_one
                if pnl_leg_two < lowest_pnl_leg_two:
                    lowest_pnl_leg_two = pnl_leg_two

                if index + candle_lives == entry.shape[0] - 1:
                    exit_pnl[index + candle_lives] = pnl_leg_one + pnl_leg_two
                    break
                candle_lives += 1

            realized_pnl[index + candle_lives] = pnl_leg_one + pnl_leg_two

    return unrealized_pnl, realized_pnl, exit_pnl, lowest_pnl_leg_one, lowest_pnl_leg_two   

In [8]:
vols_bids = wrapper.get_historical_risk_reversal_by_delta_and_tenor(bucket, 'volatility',
                                                                                       start,
                                                                                       end,
                                                                                       delta, tenor, field='bid_iv', normalize_by_ATM=normalize_by_ATM, timeframe=timeframe)
vols_asks = wrapper.get_historical_risk_reversal_by_delta_and_tenor(bucket, 'volatility',
                                                                                       start,
                                                                                       end,
                                                                                       delta, tenor, field='ask_iv', normalize_by_ATM=normalize_by_ATM, timeframe=timeframe)


func:'get_historical_risk_reversal_by_delta_and_tenor' took: 22.2577 sec
func:'get_historical_risk_reversal_by_delta_and_tenor' took: 20.8066 sec


(array([nan, nan, nan, ..., nan, nan, nan]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 0,
 0)

In [10]:
backtest = RunBacktest_RR(entries, vols_bids, vols_asks, exits, 0, 0, 0)
backtest

(array([nan, nan, nan, ..., nan, nan, nan]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 0,
 0)